In [1]:
import os
import json
from IPython.display import Image, display

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.util.tools_fabric import ToolsFabric

from react_agent.src.util.llm_proxy import LLM_PROXY

from experiments.util.fact_score.fact_scorer import FactScorer

In [2]:
USE_MCP = False
fact_scorer = FactScorer()
path_to_ressources = "./ressources/"

files = {
    "extensibility": "extensibility_assistance_result.json",
    "malaysia": "malaysia_support_result.json",
    "peppol": "peppol_support_result.json",
}

In [3]:
file_path = os.path.abspath(path_to_ressources + files["extensibility"])


with open(file_path, encoding="utf8") as f:
    json_file = json.load(f)

In [ ]:
case = json_file["cases"][1]

Case ID:  EA-2


In [5]:
triage_agent = Triage()

query_categorization = triage_agent.triage_user_message(
    user_message=case["question"]
)
print(query_categorization)

{'user_query': 'For a Greece customer in Public Cloud, what are the steps to enhance the generated XML before sending it to the Tax Authorities using ADT?', 'category': 'Knowledge-QA'}


In [6]:
tools = ToolsFabric.get_tools_for_category( use_mcp = USE_MCP, configuration = query_categorization["category"] )

In [7]:
if USE_MCP:
    async with MultiServerMCPClient( tools ) as client:
        agent = ReActAgent(tool_list=client.get_tools())
        await agent.arun_and_print_agent_stream(
            user_message=query_categorization["user_query"]
        )
else:
    agent = ReActAgent(tool_list=tools)
    agent.run_and_print_agent_stream(user_message=query_categorization["user_query"])

================================ Human Message =================================

For a Greece customer in Public Cloud, what are the steps to enhance the generated XML before sending it to the Tax Authorities using ADT?
================================== Ai Message ==================================

**Observation:** The user is asking for the steps to enhance the generated XML for a Greece customer in Public Cloud before sending it to the Tax Authorities using ADT (ABAP Development Tools).

**Thought:** To address this request, I need to provide a detailed procedure for enhancing XML documents using ADT in the context of SAP Document and Reporting Compliance. This involves understanding the XML generation process, the enhancement capabilities within ADT, and any specific compliance requirements for Greece.

**Action Plan:**
1. Retrieve information on enhancing XML documents using ADT.
2. Gather specific compliance requirements for Greece related to XML submissions.
3. Compile the ste

In [8]:
agent_answer = agent.response_message.content

In [9]:
output = await fact_scorer.classify_facts_in_context(facts=case.get("facts"), knowledge_source=agent_answer)

for result in output:
    print(f"Fact: {result.fact}")
    print(f"Contained: {result.is_contained}")
    print(f"Reason: {result.reason}")
    print("-" * 40)

Fact: The task involves enhancing an XML file for a Greece customer in Public Cloud.
Contained: True
Reason: The context explicitly mentions enhancing the generated XML for a Greece customer in Public Cloud before sending it to the Tax Authorities using ADT.
----------------------------------------
Fact: The enhancement is done using ABAP Development Tools (ADT).
Contained: True
Reason: The context explicitly mentions using ADT (Eclipse-based ABAP Development Tools) to create enhancement BAdI implementations for XML mappings.
----------------------------------------
Fact: The enhancement spot used is ES_EDOCUMENT_CLOUD.
Contained: False
Reason: The context does not mention the enhancement spot ES_EDOCUMENT_CLOUD.
----------------------------------------
Fact: A new BAdI Enhancement Implementation is created by right-clicking ES_EDOCUMENT_CLOUD.
Contained: False
Reason: The context mentions using ADT to create enhancement BAdI implementations but does not specify the action of right-cli

In [10]:
LLM_PROXY.print_usage()

Call Count: 14
Token Usage: {'input_tokens': 25055, 'output_tokens': 2294, 'total_tokens': 27349}
